In [2]:
import os,sys
import pandas as pd
import numpy as np
os.chdir('/data/arpit.goel/31_PricePrediction')
pd.set_option('display.float_format', lambda x: '%.1f' % x)

df_raw_train=pd.read_csv('01.RawData/train.tsv',delimiter='\t')
df_raw_test=pd.read_csv('01.RawData/test.tsv',delimiter='\t')

IOError: File 01.RawData/train.tsv does not exist

In [139]:
cat_price=df_raw_train.groupby(['category_name'])['price']
cat_price=cat_condition_price.apply(lambda x: x.describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])).unstack()
cat_price.to_csv('02.Profile/1.CatConditionPriceDistribution.csv')
df_raw_train['price_lower_bound']=df_raw_train.category_name.map(cat_price['5%']).fillna(5)
df_raw_train['price_upper_bound']=df_raw_train.category_name.map(cat_price['95%']).fillna(75)
df_raw_train['price_clean']=np.clip(df_raw_train['price'],df_raw_train['price_lower_bound'],df_raw_train['price_upper_bound'])
cat_price_clean=df_raw_train.groupby(['category_name'])['price_clean']
cat_price_clean=cat_price_clean.apply(lambda x: x.describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])).unstack()


In [155]:
cat_condition_price=df_raw_train.groupby(['category_name','item_condition_id'])['price_clean'].mean().unstack()
cat_condition_price=cat_condition_price.apply(lambda x: x.describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])).unstack()
cat_condition_price.to_csv('02.Profile/2.CatConditionPriceDistribution.csv')

In [144]:
df_raw_test['price']=df_raw_test.category_name.map(cat_price_clean['mean']).fillna(25)
df_raw_test[['test_id','price']].to_csv('03.Submissions/01.CategoryMean.csv',index=False)

In [242]:
cat_price=df_raw_train.groupby('category_name')['price'].apply(lambda x: {'sum':x.sum(),'count':x.size}).unstack().reset_index()
cat_price['category0']=cat_price['category_name'].map(lambda x: str(x).split('/')[0]).map(clean_category)
cat_price['category1']=cat_price['category_name'].map(lambda x: str(x).split('/')[1] if str(x).count('/')>=1 else '').map(clean_category)
cat_price['category2']=cat_price['category_name'].map(lambda x: str(x).split('/')[2] if str(x).count('/')>=2 else '').map(clean_category)

In [258]:
cat_price.to_csv('02.Profile/3.CatTypes.csv')

In [257]:
(df_raw_train['brand_name'].value_counts()>1000).value_counts()

False    4667
True      142
Name: brand_name, dtype: int64

In [252]:
df_raw_train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,price_lower_bound,price_upper_bound,price_clean,clean_cat,category0,category1,category2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.00000,1,No description yet,7.00000,46.00000,10.00000,t-shirt top men,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.00000,0,This keyboard is in great condition and works ...,7.00000,133.70000,52.00000,compon electron part comput tablet,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.00000,1,Adorable top with a hint of lace and a key hol...,6.00000,35.00000,10.00000,top blous women,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.00000,1,New with tags. Leather horses. Retail for [rm]...,6.00000,55.85000,35.00000,home decor accent,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.00000,0,Complete with certificate of authenticity,3.00000,94.00000,44.00000,necklac jewelri women,Women,Jewelry,Necklaces


Freq, Average
Categories: Taken 1,2,3 at a time. 3+3+1
Brand: Taken single
Length of description
Length of name
condition Type
shipping
Cost present in description
Text mine presence of color
Text mine ML/L/Oz/KG

In [241]:
import string
to_space= '''&,()+'"/'''
trans=string.maketrans(to_space, ' '*len(to_space))

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

def clean_category(x):
    if len(x)==0:
        return ''
    x=re.sub(chr(195),'e',x)
    x=re.sub(chr(169),'',x)
    x=x.translate(trans)
    x=re.sub(' +',' ',x)
    x=x.lower()
    x=' '.join(list(set(map(ps.stem,x.split(' ')))))
    x=re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '',x)
    return x

all_cat_counts=df_raw_train.groupby('category_name')['price_clean']
all_cat_counts=all_cat_counts.apply(lambda x: {'mean_price':x.mean(),'count':x.size}).unstack().reset_index()
all_cat_counts['clean_category_name']=all_cat_counts['category_name'].map(clean_category)
cat_mapping=pd.Series(all_cat_counts['clean_category_name'].tolist(),index=all_cat_counts['category_name'].tolist())
df_raw_train['clean_cat']=df_raw_train['category_name'].map(cat_mapping).fillna('other')


In [113]:
train_cat_counts=df_raw_train.category_name.value_counts().reset_index()
train_cat_counts.columns=['category_name','count_train']
test_cat_counts=df_raw_test.category_name.value_counts().reset_index()
test_cat_counts.columns=['category_name','count_test']
cat_counts=pd.merge(train_cat_counts,test_cat_counts,on='category_name',how='outer')
cat_counts[cat_counts['count_train'].isnull()]['count_test'].sum()

30.0

In [156]:
unigram_count={}
bigram_count={}
for row in all_cat_counts.iterrows():
    clean_name=row[1]['clean_category_name'].split(' ')
    for word in set(clean_name):
        unigram_count[word]=unigram_count.get(word,0)+row[1]['count']
    for word1,word2 in zip(clean_name[:-1],clean_name[1:]):
        key=word1+' '+word2
        bigram_count[key]=bigram_count.get(key,0)+row[1]['count']
    
vocabulary=[key for key,value in unigram_count.items() if value>1000 and value<20000]
vocabulary+=[key for key,value in bigram_count.items() if value>1000 and value<20000]



In [228]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Lasso
vectorizer = CountVectorizer(vocabulary=vocabulary)
X=vectorizer.fit_transform(all_cat_counts['clean_category_name'])
reg=Lasso(alpha=0.1)
reg.fit(X.todense(),all_cat_counts['mean_price'])
coef=pd.Series(reg.coef_,index=vocabulary)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
coef.sort_values()
#print reg.intercept_

book                  -4.29755
bath                  -1.68164
paper                 -0.90000
nurseri                0.00000
digit photographi      0.00000
coat vest              0.00000
tool makeup            0.00000
g-string thong         0.00000
comput laptop          0.00000
hat accessori          0.00000
mo t-shirt             0.00000
0-24 one-piec          0.00000
sunglass women         0.00000
beverag cooler         0.00000
beauti women           0.00000
protector accessori    0.00000
boy shoe               0.00000
storag jewelri         0.00000
skirt apparel          0.00000
photo electron         0.00000
mlb sport              0.00000
shirt athlet           0.00000
coat jean              0.00000
anim toy               0.00000
comput tablet          0.00000
other daili            0.00000
blous halter           0.00000
shoulder s             0.00000
motorcycl women        0.00000
men athlet             0.00000
                        ...   
men jersey             0.00000
action c

In [220]:
df_raw_train['clean_cat'].describe()
#cat_mapping

count                                 1476208
unique                                   1278
top       pant leg tight apparel athlet women
freq                                    60177
Name: clean_cat, dtype: object

In [221]:
df_raw_train.describe()

,train_id,item_condition_id,price,shipping,price_lower_bound,price_upper_bound,price_clean
count,1482535.00000,1482535.00000,1482535.00000,1482535.00000,1482535.00000,1482535.00000,1482535.00000
mean,741267.00000,1.90738,26.73752,0.44727,7.12295,68.75210,24.71331
std,427971.13500,0.90316,38.58607,0.49721,2.58184,51.85957,26.47087
min,0.00000,1.00000,0.00000,0.00000,3.00000,4.00000,3.00000
25%,370633.50000,1.00000,10.00000,0.00000,5.00000,40.00000,10.00000
50%,741267.00000,2.00000,17.00000,0.00000,7.00000,55.85000,17.00000
75%,1111900.50000,3.00000,29.00000,1.00000,9.00000,78.00000,29.00000
max,1482534.00000,5.00000,2009.00000,1.00000,195.00000,693.60000,693.60000
